# Passo 1: Baixar Dados

Aqui usaremos uma planilha do Microsoft Excel com dados de compras de usuários em um sistema de varejo. O nome do arquivo é `retail.xlsx`.

In [ ]:
!gdown https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n

Downloading...
From: https://drive.google.com/uc?id=1NK-2z0l-qTplDJJ2SHpTVBGRP3zWAK-n
To: /content/retail.xlsx
23.7MB [00:00, 111MB/s] 


Usaremos a biblioteca `pandas` para fazer a leitura dos dados

In [ ]:
import pandas as pd

Aqui fazemos a leitura dos dados para um objeto `DataFrame` do `pandas`.

In [ ]:
# Esta linha de código pode demorar cerca de 1 min para rodar
df = pd.read_excel('retail.xlsx')

Agora podemos dar uma observada nos dados para entender como estão organizados.

In [ ]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


A tabela acima mostra a estrutura da planilha. Podemos observar os campos:

- **InvoiceNo**: Este é um identificador único para cada compra.
- **StockCode**: Identificador único para cada produto.
- **Description**: Descrição do produto.
- **Quantidade**: Quantidade daquele produto, naquela compra.
- **InvoiceDate**: Dia e hora da compra.
- **CustomerID**: Identificador único do cliente.

In [ ]:
df.shape

(541909, 8)

Acima vemos que essa base de dados possui 541.909 registros. Cada registro representa uma linha, ou seja, um produto comprado (cuja quantidade pode ser maior que 1). Diferentes linhas podem representar diferentes itens de uma mesma compra.

# Passo 2: Remoção de Dados Nulos

Neste passo vamos remover da base de dados os dados relativos a compras de produtos onde algum dos dados de interesse estejam faltando.

Começamos observando a quantidade de dados nulos

In [ ]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Acima podemos observar há 1.454 registros sem a descrição do produto, e 135.080 registros sem a identificação do cliente comprador.

No código abaixo descartaremos esses dados que aparecem com algum campo nulo.

In [ ]:
# Remove dados nulos
df.dropna(inplace=True)
# Verifica novamente
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

# Passo 3: Dicionário de Produtos

In [ ]:
# Separa apenas as colunas de código de produto e descrição
products = df[["StockCode", "Description"]]

# Remove duplicados
products.drop_duplicates(inplace=True, subset='StockCode', keep="last")

# create product-ID and product-description dictionary
products_dict = products.groupby('StockCode')['Description'].apply(list).to_dict()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# Teste do dicionário
products_dict['84029G'][0]

'KNITTED UNION FLAG HOT WATER BOTTLE'

# Passo 4: Preparação dos Dados

Neste passo vamos preprarar os dados transformando o histórico de compras de cada consumidor numa espécie de "frase", onde cada "palavra" é um produto comprado.

Começamos convertendo os códigos de produto (coluna **StockCode**) para string, para usar como "palavras" no treinamento de um modelo word2vec no `gensim` mais tarde.

In [ ]:
df['StockCode'] = df['StockCode'].astype(str)

Agora vamos confirmar quantos clientes únicos temos nessa base de dados

In [ ]:
# A linha abaixo cria uma lista coletando os ids
# da coluna CustomerID, selecionando apenas ids
# únicos (não recolhe ids repetidos)
customers = df["CustomerID"].unique().tolist()
len(customers)

4372

Acima verificamos que há 4.372 clientes na base de dados. Para cada um desses clientes vamos verificar o histórico de compras, criando 4.372 sequências de compras.

No código abaixo embaralhamos a ordem dos ids dos clientes na lista `customers`.

In [ ]:
import random
random.shuffle(customers)

Agora vamos separar dados de treinamento e de validação

In [ ]:
# Calcula a quantidade de clientes que usaremos
# para treinamento
train_size = int(0.9 * len(customers))

# Separa os consumidores em duas listas: uma para
# treinamento e a outra para validação
customers_train = customers[:train_size]
customers_val = customers[train_size:len(customers)]

# Baseados nessa separação acima, separamos os dados
df_train = df[df['CustomerID'].isin(customers_train)]
df_val = df[df['CustomerID'].isin(customers_val)]

Com essas listas de clientes criamos abaixo as sequências de compras de acordo com os históricos de cada cliente.

In [ ]:
# Esse módulo serve para mostrar uma barra de progresso
def compile_orders(customers, df):
  ''' Essa função coleta todas compras do histódico
      de cada cliente. O parâmetro customers é a lista
      de ids de clientes e o parâmetro df é o objeto
      DataFrame do pandas com os dados de cada compra.
      O valor retornado é uma lista de listas, onde cada
      lista interna contém a seguência de códigos de produto
      de cada compra, na ordem que se apresentava no
      histórico.
  '''
  orders = []
  for customer in customers:
    order = df[df['CustomerID'] == customer]['StockCode'].tolist()
    orders.append(order)
  return orders

In [ ]:
# Aqui separamos as listas de listas de compras. Este código
# demora cerca de 1 minuto para rodar
orders_train = compile_orders(customers_train, df_train)
orders_val = compile_orders(customers_val, df_val)

# Word2Vec


## Treinando o modelo

In [ ]:
from gensim.models import Word2Vec


text_word2vec = Word2Vec(orders_train, size=100, window=5, min_count=1, workers=5)



## Procurando produtos parecidos

In [ ]:

translator = Translator()

def find_similar_prod(model,products_dict, code_product):
  
  try:
    prod_search = products_dict[code_product]
  except:
    prod_search = products_dict[int(code_product)]

  vals = text_word2vec.wv.similar_by_word(code_product)

  print('Produto procurado ', str(prod_search), '\n')

  print('Produtos similiares: \n')
  for i in range(len(vals)):
    try:
      prod_find = products_dict[vals[i][0]]
    except:
      prod_find = products_dict[int(vals[i][0])]
    print(str(prod_find), '- Taxa de similiaridade = ', vals[i][1]*100, '% \n')
    K=0

  return None

In [ ]:
n = 210
find_similar_prod(text_word2vec,products_dict, df['StockCode'][n])

Produto procurado  ['SMALL HEART MEASURING SPOONS'] 

Produtos similiares: 

['LARGE HEART MEASURING SPOONS'] - Taxa de similiaridade =  96.66997790336609 % 

['FUNKY WASHING UP GLOVES ASSORTED'] - Taxa de similiaridade =  88.3884608745575 % 

['TOAST ITS - I LOVE YOU '] - Taxa de similiaridade =  85.85157990455627 % 

['SET OF 3 HEART COOKIE CUTTERS'] - Taxa de similiaridade =  84.80281829833984 % 

['36 FOIL HEART CAKE CASES'] - Taxa de similiaridade =  84.7124457359314 % 

['RETROSPOT RED WASHING UP GLOVES'] - Taxa de similiaridade =  84.18302536010742 % 

['SET OF 3 BUTTERFLY COOKIE CUTTERS'] - Taxa de similiaridade =  83.54165554046631 % 

['BAKING MOULD HEART MILK CHOCOLATE'] - Taxa de similiaridade =  82.67738819122314 % 

['BAKING MOULD CHOCOLATE CUPCAKES'] - Taxa de similiaridade =  82.29409456253052 % 

['IVORY PAPER CUP CAKE CASES '] - Taxa de similiaridade =  81.95675611495972 % 



/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## Procurando por lista

In [ ]:
def search_by_list(model, order_list ,products_dict):
  mean_vector =  np.zeros((model.vector_size))
  
  print('Produtos da lista: \n')
  count = 0 
  for product in order_list:
    if product in model.wv:
      count = count+1
      mean_vector = mean_vector + model[product]
      try:
        print(str(products_dict[product])[2:-2] , end = ', ')
      except:
        print(str(products_dict[int(product)])[2:-2], end =', ')
      if count == 4:
        count = 0
        print('\n')
  mean_vector = mean_vector/model.vector_size
  prod_find = model.similar_by_vector(mean_vector)
  count = 0
  print('\n')
  print(' Produtos sugeridos: \n')
  for desc, val in prod_find:
    if desc in order_list:
      pass
    else:
      count = count+1
      try:
        print(str(products_dict[desc])[2:-2], '| Taxa de similiaridade = ', val*100, end =  '')
      except:
        print(str(products_dict[int(desc)])[2:-2], '| Taxa de similiaridade = ', val*100, end = ', ')
    if count==2:
      count = 0
      print('\n')

  print('\n \n')
  return None

In [ ]:
search_by_list(text_word2vec, orders_train[1],products_dict)

Produtos da lista: 



 Produtos sugeridos: 

MEDIUM CHINESE STYLE SCISSOR | Taxa de similiaridade =  0.0, TROPICAL PASSPORT COVER  | Taxa de similiaridade =  0.0, 

FOLDING UMBRELLA WHITE/RED POLKADOT | Taxa de similiaridade =  0.0, ZINC SWEETHEART SOAP DISH | Taxa de similiaridade =  0.0, 

COLUMBIAN CANDLE ROUND | Taxa de similiaridade =  0.0, BEADED PEARL HEART WHITE ON STICK | Taxa de similiaridade =  0.0

NUMBER TILE VINTAGE FONT 3 | Taxa de similiaridade =  0.0, DOLLY HONEYCOMB GARLAND | Taxa de similiaridade =  0.0, 

SET OF 2 TEA TOWELS PING MICROWAVE | Taxa de similiaridade =  0.0, RED RETRO KITCHEN WALL CLOCK | Taxa de similiaridade =  0.0, 


 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).


## Testando os modelos com os dados de validação

In [ ]:
idx = np.random.randint(1, high = len(orders_val))
search_by_list(text_word2vec, orders_val[idx],products_dict)

Produtos da lista: 

ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKELIKE RED , BOUDOIR SQUARE TISSUE BOX, PINK PAISLEY SQUARE TISSUE BOX , 

RED RETROSPOT TISSUE BOX, SET/20 RED RETROSPOT PAPER NAPKINS , CARD MOTORBIKE SANTA, CARD CHRISTMAS VILLAGE, 

RED DAISY POCKET BOOK , RED DAISY POCKET BOOK , PENS ASSORTED FUNNY FACE, ASS FLORAL PRINT MULTI SCREWDRIVER, 

ROLL WRAP 50'S CHRISTMAS, ROLL WRAP 50'S RED CHRISTMAS , SET OF 3 GOLD FLYING DUCKS, RECYCLED ACAPULCO MAT GREEN, 



 Produtos sugeridos: 

EGG CUP HENRIETTA HEN CREAM  | Taxa de similiaridade =  89.82246518135071, POPPY FIELDS CHOPPING BOARD | Taxa de similiaridade =  88.94945979118347, 

SNACK TRAY HAPPY FOREST   | Taxa de similiaridade =  88.1838321685791, IVORY RETRO KITCHEN WALL CLOCK | Taxa de similiaridade =  88.18351030349731, 

SNACK TRAY PAISLEY PARK | Taxa de similiaridade =  88.11575174331665, RED APPLES CHOPPING BOARD    | Taxa de similiaridade =  88.08581233024597, 

SET OF 6 RIBBONS PARTY | Taxa de similiaridade =  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
idx = np.random.randint(1, high = len(orders_val))
search_by_list(text_word2vec, orders_val[idx],products_dict)

Produtos da lista: 

PLASTERS IN TIN VINTAGE PAISLEY , PLASTERS IN TIN STRONGMAN, BIRD DECORATION RED RETROSPOT, MEASURING TAPE BABUSHKA PINK, 

BIRDS MOBILE VINTAGE DESIGN, MOBILE VINTAGE HEARTS , GUMBALL COAT RACK, RETROSPOT WOODEN HEART DECORATION, 

RED RETROSPOT PEG BAG, DOLLY GIRL LUNCH BOX, SPACEBOY LUNCH BOX , PENCIL CASE LIFE IS BEAUTIFUL, 

MEMO BOARD RETROSPOT  DESIGN, RED RETROSPOT WRAP , BLUE POLKADOT WRAP, PINK POLKADOT WRAP , 

PINK PAISLEY ROSE GIFT WRAP, BLUE SCANDINAVIAN PAISLEY WRAP, SET/10 RED POLKADOT PARTY CANDLES, SET OF 4 KNICK KNACK TINS DOILY , 

STORAGE TIN VINTAGE DOILY , SET OF 3 WOODEN HEART DECORATIONS, SET OF 3 WOODEN STOCKING DECORATION, VINTAGE RED TRIM ENAMEL BOWL , 

VINTAGE RED ENAMEL TRIM MUG , RED RETROSPOT SHOPPER BAG, POCKET BAG BLUE PAISLEY RED SPOT, ADVENT CALENDAR GINGHAM SACK, 

JAM MAKING SET PRINTED, MEASURING TAPE BABUSHKA BLUE, 6 RIBBONS RUSTIC CHARM, RIBBON REEL HEARTS DESIGN , 

OPEN CLOSED METAL SIGN, BAKING SET 9 PIECE RETROSPOT , RE

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
idx = np.random.randint(1, high = len(orders_val))
search_by_list(text_word2vec, orders_val[idx],products_dict)

Produtos da lista: 

ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKELIKE IVORY, HAND WARMER OWL DESIGN, HAND WARMER BIRD DESIGN, 

HAND WARMER UNION JACK, CREAM HEART CARD HOLDER, COLOURED GLASS STAR T-LIGHT HOLDER, GREEN CHRISTMAS TREE CARD HOLDER, 

HAND OVER THE CHOCOLATE   SIGN , ANTIQUE SILVER T-LIGHT GLASS, HOME BUILDING BLOCK WORD, WOOD BLACK BOARD ANT WHITE FINISH, 

HAND WARMER OWL DESIGN, CREAM HANGING HEART T-LIGHT HOLDER, ALARM CLOCK BAKELIKE GREEN, PLEASE ONE PERSON METAL SIGN, 

ALARM CLOCK BAKELIKE PINK, SET OF 3 CAKE TINS PANTRY DESIGN , IVORY KITCHEN SCALES, RED KITCHEN SCALES, 

ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKELIKE RED , ALARM CLOCK BAKELIKE IVORY, ROUND CAKE TIN VINTAGE GREEN, 

RETROSPOT HEART HOT WATER BOTTLE, RED HANGING HEART T-LIGHT HOLDER, CREAM HEART CARD HOLDER, WOOD BLACK BOARD ANT WHITE FINISH, 

CHICK GREY HOT WATER BOTTLE, HOME BUILDING BLOCK WORD, RED STRIPE CERAMIC DRAWER KNOB, BLUE STRIPE CERAMIC DRAWER KNOB, 

BLUE SPOT CERAMIC DRAWER KNOB, R

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
idx = np.random.randint(1, high = len(orders_val))
search_by_list(text_word2vec, orders_val[idx],products_dict)

Produtos da lista: 

SAVE THE PLANET MUG, 

 Produtos sugeridos: 

HOME SWEET HOME MUG | Taxa de similiaridade =  97.78735637664795, VINTAGE BILLBOARD LOVE/HATE MUG | Taxa de similiaridade =  95.74099779129028, 

KINGS CHOICE MUG | Taxa de similiaridade =  95.25653123855591, GLAMOROUS  MUG | Taxa de similiaridade =  95.09186744689941, 

RETRO COFFEE MUGS ASSORTED | Taxa de similiaridade =  94.97313499450684, GIN AND TONIC MUG | Taxa de similiaridade =  93.26659440994263, 

POTTERING MUG | Taxa de similiaridade =  92.06171035766602, VINTAGE BILLBOARD DRINK ME MUG | Taxa de similiaridade =  92.06134080886841, 

IF YOU CAN'T STAND THE HEAT MUG | Taxa de similiaridade =  90.52301049232483, 
 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
idx = np.random.randint(1, high = len(orders_val))
search_by_list(text_word2vec, orders_val[idx],products_dict)

Produtos da lista: 

JUMBO  BAG BAROQUE BLACK WHITE, JUMBO BAG STRAWBERRY, JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG PINK VINTAGE PAISLEY, 

JUMBO BAG SCANDINAVIAN BLUE PAISLEY, JUMBO BAG VINTAGE DOILY , JUMBO BAG APPLES, GIN AND TONIC DIET METAL SIGN, 

MINI JIGSAW DOLLY GIRL, MINI JIGSAW BUNNIES, MINI JIGSAW LEAP FROG, MINI JIGSAW SPACEBOY, 

MINI JIGSAW GO TO THE FAIR, MAGIC DRAWING SLATE DOLLY GIRL , MAGIC DRAWING SLATE SPACEBOY , MAGIC DRAWING SLATE BAKE A CAKE , 

EDWARDIAN PARASOL NATURAL, EDWARDIAN PARASOL RED, SET OF 6 SOLDIER SKITTLES, 4 TRADITIONAL SPINNING TOPS, 

SMALL HEART MEASURING SPOONS, 3D SHEET OF CAT STICKERS, 3D SHEET OF DOG STICKERS, ZINC METAL HEART DECORATION, 

PARTY BUNTING, SPOTTY BUNTING, FIVE CATS HANGING DECORATION, BIRDS MOBILE VINTAGE DESIGN, 

DOLLY GIRL BABY GIFT SET,  SPACEBOY BABY GIFT SET, GIRLS VINTAGE TIN SEASIDE BUCKET, BOYS VINTAGE TIN SEASIDE BUCKET, 

SET OF 4 KNICK KNACK TINS DOILY , TREASURE TIN BUFFALO BILL , ASSORTED COLOUR BIRD ORNAME

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# Testando a acertividade nas indicações do modello


In [ ]:
def search_by_prod(model, order_list ,products_dict):
  mean_vector =  np.zeros((model.vector_size))
  
  acc = dict()
  for product in order_list:
    if product in model.wv:
      find_prod = text_word2vec.wv.similar_by_word(product)
      test_list = order_list
      test_list.remove(product)
      k = np.array(test_list)
      try:
        print(products_dict[product],'\n')
      except:
        print(products_dict[int(product)],'\n')
      codes, similiar_val = list(zip(*find_prod))
      if product in codes:
        acc[product] = (sum((k==codes).astype(float)))
      else:
        acc[product] = 0
      try:
        print((k==codes),'\n')
      except:
        pass
  return acc